# Calculate pO2 using T_bar, S_bar, O2_bar

In [1]:
import xarray as xr
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
USER = os.environ['USER']
var1 = 'O2' # mmol/m3
var2 = 'TEMP' # deg C
var3 = 'SALT' # PSS
var = [var1,var2,var3]
tres = 'monthly'
#tres = 'yearly'

### DPLE hindcast of T,S,O2

In [3]:
path = f'/glade/scratch/{USER}/DPLE-results'
filename = f'{path}/DPLE-FOSI_hindcast_T_S_O2_1948-2017_{tres}.nc'
ds_hind = xr.open_dataset(filename)
ds_hind

<xarray.Dataset>
Dimensions:  (z_t: 35, nlat: 140, nlon: 195, time: 840)
Coordinates:
  * z_t      (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.277e+04 5.794e+04
    TLAT     (nlat, nlon) float64 ...
    TLONG    (nlat, nlon) float64 ...
    ULAT     (nlat, nlon) float64 ...
    ULONG    (nlat, nlon) float64 ...
  * time     (time) object 0249-02-01 00:00:00 ... 0319-01-01 00:00:00
Dimensions without coordinates: nlat, nlon
Data variables:
    O2       (time, z_t, nlat, nlon) float32 ...
    TEMP     (time, z_t, nlat, nlon) float32 ...
    SALT     (time, z_t, nlat, nlon) float32 ...

### calculate climatological-mean T or O2

In [4]:
dsf = xr.Dataset()
dsf['O2_mean'] = ds_hind['O2'].mean(dim='time')
dsf['TEMP_mean'] = ds_hind['TEMP'].mean(dim='time')
dsf['SALT_mean'] = ds_hind['SALT'].mean(dim='time')

### calculate 3d depth

In [5]:
depth = ds_hind['z_t']*dsf['O2_mean']/dsf['O2_mean']/1e2 # m

In [6]:
dsf['3ddepth'] = depth.transpose("z_t","nlat","nlon")

### save to file

In [7]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dsf.load()

<xarray.Dataset>
Dimensions:    (z_t: 35, nlat: 140, nlon: 195)
Coordinates:
  * z_t        (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.277e+04 5.794e+04
    TLAT       (nlat, nlon) float64 14.96 14.96 14.97 ... 70.43 70.42 70.42
    TLONG      (nlat, nlon) float64 101.2 102.3 103.4 ... 318.6 319.2 319.7
    ULAT       (nlat, nlon) float64 15.13 15.13 15.14 ... 70.64 70.63 70.63
    ULONG      (nlat, nlon) float64 101.7 102.9 104.0 ... 318.9 319.5 320.0
Dimensions without coordinates: nlat, nlon
Data variables:
    O2_mean    (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    TEMP_mean  (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    SALT_mean  (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan
    3ddepth    (z_t, nlat, nlon) float32 nan nan nan nan nan ... nan nan nan nan

In [8]:
%%time
dsf.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_O2_T_S_mean_dimension_reduced_1948-2017_{tres}.nc', mode='w')

CPU times: user 23.7 ms, sys: 11.2 ms, total: 34.9 ms
Wall time: 83.2 ms
